# Dask Array

<img src="http://dask.readthedocs.io/en/latest/_images/dask_horizontal.svg"
     align="right"
     width="30%"
     alt="Dask logo\">


Dask array provides a parallel, larger-than-memory, n-dimensional array using blocked algorithms. Similar to how `dask.dataframe` mirrors the `pandas` interface, `dask.array` mirrors the `numpy` interface.

In this notebook we'll briefly look at the `dask.array` interface, and then build understanding by looking at Dask graphs for various operations.

For more information on `dask.array`, see the documentation: https://docs.dask.org/en/latest/array.html

## Basics

### Create random dataset

In [ ]:
import dask
import dask.array as da

x = da.random.random((2000, 2000), chunks=(1000, 1000))

### Inspect `dask.array.Array` object

In [ ]:
x

In [ ]:
x.dtype

In [ ]:
x.shape

In [ ]:
x.chunks

In [ ]:
# All methods on a dask array
[n for n in dir(x) if not n.startswith('_')]

In [ ]:
# All functions in the dask array namespace
[n for n in dir(da) if not n.startswith('_')]

## Inner Workings

<img src="https://docs.dask.org/en/latest/_images/dask-array-black-text.svg" width="50%">

Dask array breaks large arrays into a bunch of smaller arrays along each axis. Familiar operations (like `.mean()`) are then built out of blocked algorithms allowing for parallel/out-of-core computation.

Here we'll inspect the graphs of increasingly complex operations to try and see what's going on.

### Random array

In [ ]:
x = da.random.random((2000, 2000), chunks=(1000, 1000))

In [ ]:
x.visualize()

### Elementwise operations

In [ ]:
da.sin(x).visualize()

In [ ]:
(da.sin(x) + 1).visualize()

### Visualizing optimizations

When you call `.compute()`, Dask takes the underlying graph and optimizes it before starting computation. By default `.visualize()` just visualizes the raw graph *before* optimization. To see the optimized graph use `.visualize(optimize_graph=True)`.

In [ ]:
(da.sin(x) + 1).visualize(optimize_graph=True)

### Mean of this array

In [ ]:
x.mean().visualize()

### Mean along an axis

In [ ]:
x.mean(axis=1).visualize()

### Slicing of an array

In [ ]:
x[:500, :500].visualize()

In [ ]:
x[:500, :500].visualize(optimize_graph=True)

### Dot product

In [ ]:
x.dot(x.T).visualize()

### Dot product then sum along axis

In [ ]:
x.dot(x.T).sum(axis=1).visualize()

## Exercise:

Take a few minutes to play around with the `dask.array` API, looking at the underlying graphs for common operations. What's the most complicated graph you can make? How much of a difference do optimizations make?

## User-defined Functions

Dask implements a good portion of the `numpy` API. However, sometimes there's an operation you need that isn't currently implemented. To accomodate this, `dask.array` provides a few methods for applying user-defined functions on an array:

- `da.map_blocks`: create a new array by applying a function to every block in an existing array
- `da.blockwise`: a more flexible form of `map_blocks`, supports generalized inner and outer products
- `da.map_overlap`: map a function over blocks of an array, with some overlap between blocks

### Example: `scipy.special.logit`

Dask array doesn't natively have a `logit` function, but one can be used by calling it with `map_blocks`:

In [ ]:
from scipy.special import logit

In [ ]:
x.map_blocks(logit).visualize()

### Exercise:

Compare the runtime of computing the following elementwise expression using dask builtin methods:

```
temp = da.sin(x)**2 + da.cos(x)**2
total = temp.sum()
```

with computing the same operation using a single function and a call to `map_blocks`.

```
temp = ...
blocked_total = temp.sum()
```

Which one is faster? By how much? How do their unoptimized and optimized graphs differ?

In [ ]:
total = (da.sin(x)**2 + da.cos(x)**2).sum()

In [ ]:
blocked_total = # your code here

In [ ]:
%timeit total.compute()

In [ ]:
%timeit blocked_total.compute()